In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

# quick_pp - Water Saturation Estimation

Water saturation estimation is crucial in petrophysics for several reasons:

1. **Hydrocarbon Volume Calculation**: It helps determine the volume of hydrocarbons in place. Accurate water saturation (Sw) values are essential for calculating the original oil in place (OOIP) and original gas in place (OGIP) volumes¹(https://petroshine.com/fluid-saturation/).
2. **Reservoir Characterization**: Understanding the distribution of water saturation helps in characterizing the reservoir, which is vital for planning production strategies and enhancing recovery¹(https://petroshine.com/fluid-saturation/).
3. **Production Forecasting**: Sw values are used in reservoir models to predict future production and to evaluate the economic viability of the reservoir²(https://www.mdpi.com/2077-1312/9/6/666).

### Methods to Estimate Water Saturation

1. **Resistivity Logs**: This is the most common method, where water saturation is estimated using resistivity measurements from well logs. The Archie equation is often used for clean sands, while modified versions like the Waxman-Smits model are used for shaly sands³(https://petrowiki.spe.org/Water_saturation_determination).
2. **Capillary Pressure Measurements**: Laboratory measurements of capillary pressure and corresponding water saturation provide detailed information about the pore structure and fluid distribution³(https://petrowiki.spe.org/Water_saturation_determination).
3. **Core Analysis**: Direct measurement of water saturation from core samples using techniques like the Dean-Stark method³(https://petrowiki.spe.org/Water_saturation_determination).
4. **Nuclear Magnetic Resonance (NMR)**: NMR logging tools can provide estimates of water saturation by measuring the response of hydrogen nuclei in the formation fluids³(https://petrowiki.spe.org/Water_saturation_determination).

This notebook estimates the water saturation using Archie equation and saturation height function based on the capillary pressure measurement.


## Log Derived Water Saturation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from quick_pp.objects import Project
from quick_pp.rock_type import calc_r35_perm

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

all_data = project.get_all_data()

focused_well = 'HW-26'
well_data = all_data[all_data.WELL_NAME == focused_well].copy()

Estimation of Rw based on formation water salinity, assuming the depths are already in True Vertical Depth Sub Sea (TVDSS).
The range of Rw used in the original paper is 0.015 to 0.03 ohm.m

Estimation of cementation factor (m) based on pickett plot.

In [ ]:
from ipywidgets import widgets, interact

from quick_pp.saturation import pickett_plot

water_wells = ['HW-5', 'HW-7', 'HW-31']
focused_data = all_data[all_data.WELL_NAME.isin(water_wells)].copy()

wells = widgets.SelectMultiple(
    options=['All'] + list(focused_data['WELL_NAME'].unique()),
    value=['All'],
    description='Wells:'
)
m = widgets.FloatSlider(
    value=2,
    min=1,
    max=3,
    step=.1,
    readout_format='.1f'
)
min_rw = widgets.FloatSlider(
    value=.01,
    min=.001,
    max=.1,
    step=.001,
    readout_format='.3f'
)
min_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.min(),
    min=focused_data.DEPTH.min(),
    max=focused_data.DEPTH.max() - 10,
    step=.1,
    readout_format='.1f'
)
max_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.max(),
    min=focused_data.DEPTH.min() + 10,
    max=focused_data.DEPTH.max(),
    step=.1,
    readout_format='.1f'
)

@interact(wells=wells, m=m, min_rw=min_rw, min_depth=min_depth, max_depth=max_depth)
def param(wells, m, min_rw, min_depth, max_depth):
    if 'All' in wells:
        data = focused_data[(focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)]
    else:
        data = focused_data[(focused_data.WELL_NAME.isin(wells)) & (focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)]
    pickett_plot(data['RT'], data['PHIT'], m=m, min_rw=min_rw)

In [ ]:
from quick_pp.saturation import estimate_temperature_gradient, estimate_rw_temperature_salinity, archie_saturation

# Debug water saturation
water_salinity = 2e5
m = 2.2

temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'imperial')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

swt = archie_saturation(well_data['RT'], rw, well_data['PHIT'], m=m)
swt = swt.clip(0, 1)

plt.figure(figsize=(15, 1.5))
plt.plot(well_data['DEPTH'], swt)
plt.ylim(0, 1.5)
plt.figure(figsize=(15, 1.5))
plt.plot(well_data['DEPTH'], rw, label='RW')
plt.yscale('log')
plt.legend()
plt.tight_layout()

# quick_pp - Saturation Height Function

### Core Data

Explain the data source,
- measurement techniques
- data qc is discussed in the next section

### Define Rock Type

Define the rock type based on FZI cut-offs from previous notebook



In [ ]:
from quick_pp.rock_type import calc_fzi, rock_typing, calc_r35
from quick_pp.core_calibration import fit_j_curve, j_xplot, leverett_j, sw_shf_leverett_j, poroperm_xplot, pc_xplot

core_data = pd.read_csv(r'data\01_raw\COSTA\HW_core_data_all.csv')
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']
core_data['PC'] = core_data['O/B Pc (psia)']
core_data['PC_RES'] = core_data['O/B Pc (psia)'] * 0.088894  # oil-brine system
core_data['SW'] = core_data['Equiv Brine Sat. (Frac.)']
core_data['SWN'] = core_data.groupby('Sample')['SW'].transform(lambda x: (x - x.min()) / (1 - x.min()))

# Filter data
conditions = (
    (core_data['K mD'] > 0)
    & (core_data['Class'] == 'Good')
    # & (core_data['PC'] <= 40)
)
ori_core_data = core_data[conditions].copy()
core_data = core_data[conditions].copy()
# core_data.drop_duplicates(subset=['CPORE', 'CPERM', 'SW'], keep='last', inplace=True)

## Winland R35 Rock Types

In [ ]:
core_data = ori_core_data.copy()
# Estimate rock types
r35 = calc_r35(core_data['CPORE'], core_data['CPERM'])
core_data['R35'] = r35
r35_cut_offs = [
    0.15, .2, .3, .4, .5, .6, .7, .8, .9, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 7, 10,
]
rock_flag = rock_typing(r35, higher_is_better=True, cut_offs=r35_cut_offs)
core_data['ROCK_FLAG'] = rock_flag

QC capillary pressure data.
Select dataset for each rock type, filtering line-crossing datasets.

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import pc_xplot, poroperm_xplot

rock_flag = widgets.SelectMultiple(
    options=['All'] + sorted(list(core_data['ROCK_FLAG'].unique())),
    value=['All'],
    description='Rock Flag:'
)

@interact(rock_flag=rock_flag)
def param(rock_flag):
    # Plot all data on poroperm plot
    poroperm_xplot(core_data['CPORE'], core_data['CPERM'])
    data = core_data[core_data.ROCK_FLAG.isin(rock_flag)] if any([l for l in rock_flag if l != 'All']) else core_data

    # Plot filtered data
    poroperm_xplot(data['CPORE'], data['CPERM'], core_group=data['Sample'])
    plt.show()
    for label, temp_df in data.groupby('Sample'):
        pc_xplot(temp_df['SW'], temp_df['PC'], label=label, ylim=(0, 40))
    plt.show()
plt.close('all')

### Leverett J

Explain Leverett J technique

In [ ]:
ift = 32
theta = 30

core_data['J'] = leverett_j(core_data['PC_RES'], ift, theta, core_data['CPERM'], core_data['CPORE'])
j_xplot(core_data['SW'], core_data['J'], core_group=core_data['ROCK_FLAG'])

### QC the Pc data

The capillary pressure measurements for each Sample are plotted on a log-log plot.
The data points should fall on a relatively straight line indicating good data quality.

Based 
select the dataset for each rock type
curve fitting

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import fit_j_curve

sample = widgets.Dropdown(
    options=core_data['Sample'].unique(),
    description='Sample:'
)
conditions = (
    (core_data['SWN'] < .99)
    & (core_data['SWN'] > .05)
)
filtered_data = core_data[conditions].copy()
@interact(sample=sample)
def param(sample):
    data = filtered_data[filtered_data['Sample'] == sample]
    a, b = fit_j_curve(data['SW'], data['J'])
    j_xplot(data['SW'], data['J'], a=a, b=b, label=f'Sample {sample}: a:{a}, b:{b}',
            core_group=data['Sample'], log_log=False)

In [ ]:
j_params = {}
for sample, data in filtered_data.groupby('Sample'):
    a, b = fit_j_curve(data['SW'], data['J'])
    j_params[sample] = (a, b)

# Assign core sample to each rock type for mapping
r35_j_params = {
    1: 83,
    2: 72,
    3: 70,
    4: 84,
    5: 67,
    6: 64,
    7: 61,
    8: 63,
    9: 52,
    10: 56,
    11: 51,
    12: 42,
    13: 43,
    14: 35,
    15: 36,
    16: 32,
    17: 23,
    18: 20,
    19: 21,
    20: 15,
    21: 8,
    22: 2,
    23: 3,
}

In [ ]:
import pprint

# Map rt_skelt_params with skelt_params key
mapped_r35_j_params = {rt: j_params.get(key) for rt, key in r35_j_params.items()}
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(mapped_r35_j_params)

In [ ]:
from ipywidgets import interact, widgets

rt = widgets.Dropdown(
    options=core_data['ROCK_FLAG'].unique(),
    value=1,
    description='Rock Type:'
)

@interact(rt=rt)
def param(rt):
    a, b = mapped_r35_j_params[rt]
    data = core_data[core_data['ROCK_FLAG'] == rt]
    j_xplot(data['SWN'], data['J'], a=a, b=b, core_group=data['Sample'], log_log=False)

In [ ]:
for rt, param in mapped_r35_j_params.items():
    a, b = param
    rock_data = core_data[core_data['ROCK_FLAG'] == rt]
    j_xplot(rock_data['SWN'], rock_data['J'], a=a, b=b,  # core_group=rock_data['Sample'],
            label=f'{rt}: a:{a}, b:{b}', log_log=False)

### Estimate Free Water Level (FWL)

In [ ]:
import numpy as np

well_data['LOG_RT'] = np.log10(well_data['RT'])
well_data['NDI_V2'] = np.log10(((2.85 - well_data['RHOB']) / 1.85) - well_data['NPHI'])
input_features = ['GR', 'NPHI', 'RHOB', 'LOG_RT', 'NDI_V2'] 

# Predict PERM
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)
r35_ml = 10**(r35_model.predict(well_data[input_features]))
well_data['PERM'] = calc_r35_perm(r35_ml, well_data['PHIT'])
well_data['ROCK_FLAG'] = rock_typing(r35_ml, higher_is_better=True, cut_offs=r35_cut_offs)

In [ ]:
from quick_pp.core_calibration import sw_shf_leverett_j, sw_shf_cuddy

ift = 32
theta = 30
ghc = .837
gw = 1.135
fwl = 8550

fwl = widgets.FloatSlider(
    value=fwl,
    min=fwl / 1.1,
    max=fwl * 1.1,
    step=1
)

@interact(fwl=fwl)
def plot(fwl):
    a = well_data['ROCK_FLAG'].map(mapped_r35_j_params).apply(lambda x: x[0])
    b = well_data['ROCK_FLAG'].map(mapped_r35_j_params).apply(lambda x: x[1])
    shf = sw_shf_leverett_j(
        well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], gw=gw, ghc=ghc,
        fwl=fwl, ift=ift, theta=theta, a=a, b=b)

    plt.figure(figsize=(20, 2))
    plt.plot(well_data['DEPTH'], swt, label='SWT')
    plt.plot(well_data['DEPTH'], shf, label='SHF')
    plt.ylim(0, 1.5)
    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], well_data['ROCK_FLAG'])
    plt.legend()

## FZI Rock Types

In [ ]:
# FZI
core_data = ori_core_data.copy()
fzi = calc_fzi(core_data['CPORE'], core_data['CPERM'])
fzi_cut_offs = [
    .15, .2, .25, .3, .35, .4, .45, .5, .6, .7, .8, .9, 1.1, 1.3, 1.5, 2, 2.5, 3, 4, 5, 7, 10
]
rock_flag = rock_typing(fzi, fzi_cut_offs, higher_is_better=True)
core_data['ROCK_FLAG'] = rock_flag

QC capillary pressure data.
Select dataset for each rock type, filtering line-crossing datasets.

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import pc_xplot, poroperm_xplot

rock_flag_widget = widgets.SelectMultiple(
    options=['All'] + sorted(list(core_data['ROCK_FLAG'].unique())),
    value=['All'],
    description='Rock Flag:'
)

@interact(rock_flag=rock_flag_widget)
def param(rock_flag):
    # Plot all data on poroperm plot
    poroperm_xplot(core_data['CPORE'], core_data['CPERM'])
    data = core_data[core_data.ROCK_FLAG.isin(rock_flag)] if any([l for l in rock_flag if l != 'All']) else core_data

    # Plot filtered data
    poroperm_xplot(data['CPORE'], data['CPERM'], core_group=data['Sample'])
    plt.show()
    for label, temp_df in data.groupby('Sample'):
        pc_xplot(temp_df['SW'], temp_df['PC'], label=label, ylim=(0, 40))
    plt.show()
plt.close('all')

### Leverett J

Explain Leverett J technique

In [ ]:
ift = 32
theta = 30

core_data['J'] = leverett_j(core_data['PC_RES'], ift, theta, core_data['CPERM'], core_data['CPORE'])
j_xplot(core_data['SW'], core_data['J'], core_group=core_data['ROCK_FLAG'])

#### QC the Pc data

The capillary pressure measurements for each Sample are plotted on a log-log plot.
The data points should fall on a relatively straight line indicating good data quality.

Based 
select the dataset for each rock type
curve fitting

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import fit_j_curve

sample = widgets.Dropdown(
    options=core_data['Sample'].unique(),
    description='Sample:'
)
conditions = (
    (core_data['SWN'] < .9)
    & (core_data['SWN'] > .1)
)
filtered_data = core_data[conditions].copy()
@interact(sample=sample)
def param(sample):
    data = filtered_data[filtered_data['Sample'] == sample]
    a, b = fit_j_curve(data['SW'], data['J'])
    j_xplot(data['SW'], data['J'], a=a, b=b, label=f'Sample {sample}: a:{a}, b:{b}',
            core_group=data['Sample'], log_log=False)

In [ ]:
j_params = {}
for sample, data in filtered_data.groupby('Sample'):
    a, b = fit_j_curve(data['SW'], data['J'])
    j_params[sample] = (a, b)

# Assign core sample to each rock type for mapping
fzi_params = {
    1: 83,
    2: 86,
    3: 68,
    4: 84,
    5: 79,
    6: 64,
    7: 49,
    8: 44,
    9: 52,
    10: 40,
    11: 24,
    12: 27,
    13: 23,
    14: 20,
    15: 14,
    16: 11,
    17: 7,
    18: 8,
    19: 10,
    20: 9,
    21: 5,
    22: 2,
    23: 3
}

In [ ]:
import pprint

# Map rt_skelt_params with skelt_params key
mapped_fzi_params = {rt: j_params.get(key) for rt, key in fzi_params.items()}
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(mapped_fzi_params)

In [ ]:
from ipywidgets import interact, widgets

rt_widget = widgets.Dropdown(
    options=sorted(core_data['ROCK_FLAG'].unique()),
    description='Rock Type:'
)

@interact(rt=rt_widget)
def param(rt):
    a, b = mapped_fzi_params[rt]
    data = core_data[core_data['ROCK_FLAG'] == rt]
    j_xplot(data['SWN'], data['J'], a=a, b=b, core_group=data['Sample'], log_log=False)

In [ ]:
for rt, param in mapped_fzi_params.items():
    a, b = param
    rock_data = core_data[core_data['ROCK_FLAG'] == rt]
    j_xplot(rock_data['SWN'], rock_data['J'], a=a, b=b,  # core_group=rock_data['Sample'],
            label=f'{rt}: a:{a}, b:{b}', log_log=False)

### Estimate Free Water Level (FWL)

In [ ]:
import numpy as np

from quick_pp.rock_type import calc_fzi_perm

fzi_cut_offs = [
    .15, .2, .25, .3, .35, .4, .45, .5, .6, .7, .8, .9, 1.1, 1.3, 1.5, 2, 2.5, 3, 4, 5, 7, 10
]

well_data['LOG_RT'] = np.log10(well_data['RT'])
well_data['NDI_V2'] = np.log10(((2.85 - well_data['RHOB']) / 1.85) - well_data['NPHI'])

input_features = ['GR', 'NPHI', 'RHOB', 'LOG_RT', 'NDI_V2']
# Predict PERM
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)
fzi_ml = 10**(fzi_model.predict(well_data[input_features]))
well_data['PERM'] = calc_fzi_perm(fzi_ml, well_data['PHIT'])
well_data['ROCK_FLAG'] = rock_typing(fzi_ml, higher_is_better=True, cut_offs=fzi_cut_offs)

In [ ]:
from quick_pp.core_calibration import sw_shf_leverett_j, sw_shf_cuddy

ift = 32
theta = 30
ghc = .837
gw = 1.135
fwl = 8550

fwl = widgets.FloatSlider(
    value=fwl,
    min=fwl / 1.1,
    max=fwl * 1.1,
    step=1
)

@interact(fwl=fwl)
def plot(fwl):
    a = well_data['ROCK_FLAG'].map(mapped_fzi_params).apply(lambda x: x[0])
    b = well_data['ROCK_FLAG'].map(mapped_fzi_params).apply(lambda x: x[1])
    shf = sw_shf_leverett_j(
        well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], gw=gw, ghc=ghc,
        fwl=fwl, ift=ift, theta=theta, a=a, b=b)

    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], swt, label='SWT')
    plt.plot(well_data['DEPTH'], shf, label='SHF')
    plt.ylim(0, 1.2)
    plt.legend()
    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], well_data['ROCK_FLAG'], label='Rock Flag')
    plt.legend()

## Plot the results

In [ ]:
from quick_pp.plotter import plotly_log
fwl = 8760

# a = well_data['ROCK_FLAG'].map(params).apply(lambda x: x[0])
# b = well_data['ROCK_FLAG'].map(params).apply(lambda x: x[1])
# shf = sw_shf_leverett_j(
#     well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], gw=gw, ghc=ghc,
#     fwl=fwl, ift=ift, theta=theta, a=a, b=b
# )
shf = 0
# Plot individual results
well_data['SWT'] = swt
well_data['SHF'] = shf
fig = plotly_log(well_data, 'ft')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()